In [7]:
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

def read_sim(sim_path,q_vname,t_vname,time_format):
    
    f = xr.open_dataset(sim_path)
    sim_irf = f[q_vname].values[:]
    sim_time = pd.to_datetime(f[t_vname].values[:])
    f.close()
    return sim_irf, sim_time

def read_obs(obs_path):
    obs = []
    obs_time = []
    with open(obs_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line and line.startswith('USGS'):
                splits = line.split()
                obs_time.append(datetime.datetime.strptime(splits[2],time_format))
                if len(splits)>3:#flow data exists
                    obs.append(float(splits[3])/35.315) #convert to m^3/s
                else:
                    obs.append(np.nan)
    obs = np.asarray(obs)
    obs_time = pd.to_datetime(obs_time)
    return obs, obs_time

def plot_hyd(t_plot, obs_plot, sim_plot, obs_fmt, sim_fmt, title_str, y_lim, ofile):

    row_num=1
    col_num=1        
    fig, ax = plt.subplots(row_num,col_num)
    fig.set_figwidth(6.5*1.2*col_num) #190mm
    fig.set_figheight(6.5*0.4*row_num) #5.61 heigh/width=3/4

    dpi_value=100
#     formatter = DateFormatter('%m/%d/%y')
    formatter = DateFormatter('%Y/%m')

    ax.plot_date(t_plot, obs_plot, obs_fmt, linewidth=0.75, markersize=0.0, alpha=0.8, label='Observation')   
    ax.plot_date(t_plot, sim_plot, sim_fmt, linewidth=0.75, markersize=0.0, alpha=0.8, label='Simulation (IRF)') 

    #axis, label, title, legend
    ax.set_title(title_str, fontsize='small', fontweight='bold')
    ax.set_xlabel('Date', fontsize='small')
    ax.set_ylabel('Flow $(m^3/s)$', fontsize='small')
    ax.set_xlim(left=t_plot[0], right=t_plot[-1])
    ax.set_ylim(top=y_lim)
    ax.xaxis.set_major_formatter(formatter)
    ax.xaxis.set_tick_params(labelsize='small')#rotation=30,
    ax.legend(loc='best', fontsize='x-small', framealpha=0.5) #loc='upper right', 

    plt.rc('xtick',labelsize='small')
    plt.rc('ytick',labelsize='small')                
    fig.tight_layout()
    fig.savefig(ofile, dpi=dpi_value)
    plt.close(fig)      
    del fig
 
    return ofile

#====================================================================================#
# Main script
# root_dir = os.getcwd()
script_dir = '/glade/u/home/hongli/github/summa_mizuroute_app/calib/tayprk/analysis'
root_dir = '/glade/u/home/hongli/work/sharp/basins/tayprk/'
raw_origin_dir = os.path.join(root_dir, 'model_origin')
raw_dir = os.path.join(root_dir, 'model')

output_dir=os.path.join(root_dir,'analysis/step2_plot_route_hydro')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
outputfile = 'hydrgraph_calib_route_w_orign.png'

calib_dir_basename = 'calib/calib_route_trial'
sim_file='best/all.nc'
# q_vname1 = 'KWTroutedRunoff'
q_vname2 = 'IRFroutedRunoff'
t_vname = 'time'
time_format='%Y-%m-%d'
obs_file=os.path.join(script_dir,'Taylor_park_09107000.txt')

plot_start_date = '2009-10-01'
plot_end_date = '2018-09-30'
plot_start_date_obj = datetime.datetime.strptime(plot_start_date, time_format)
plot_end_date_obj = datetime.datetime.strptime(plot_end_date, time_format)

calib_num = 6
objs = ['RMSE','RMSE','RMSE','KGE','KGE','KGE']
alpha = 'a'
fig_file_summary = []
fig_count = 0

#=====================================================================================#
# set y limit
sim_irf, sim_time = read_sim(os.path.join(raw_dir, 'route/output/all.nc'),q_vname2,t_vname,time_format)
sim_mask = (sim_time>=plot_start_date_obj) & (sim_time<=plot_end_date_obj)
sim_plot = sim_irf[sim_mask]

obs, obs_time = read_obs(obs_file)
obs_mask = (obs_time>=plot_start_date_obj) & (obs_time<=plot_end_date_obj)
obs_plot = obs[obs_mask]

y_lim = max(np.amax(sim_plot), np.amax(obs_plot))
del sim_irf, sim_time, sim_plot, obs, obs_time, obs_plot

for i in range(calib_num):
# for i in [1]:
    calib_dir = os.path.join(root_dir,calib_dir_basename+str(i+1))    
    sim_irf, sim_time = read_sim(os.path.join(calib_dir, sim_file),q_vname2,t_vname,time_format)
    sim_mask = (sim_time>=plot_start_date_obj) & (sim_time<=plot_end_date_obj)
    sim_plot = sim_irf[sim_mask]
    
    y_lim = max(y_lim, np.amax(sim_plot))
    del sim_irf, sim_time, sim_plot 
y_lim = y_lim*1.1

#====================================================================================#
# raw results (no calibration, before Andy's manual modification to summa)
raw_outputfile = 'raw_origin.png'
ofile = os.path.join(output_dir,raw_outputfile)
print('raw_origin')

sim_irf, sim_time = read_sim(os.path.join(raw_origin_dir, 'route/output/all.nc'),q_vname2,t_vname,time_format)
sim_mask = (sim_time>=plot_start_date_obj) & (sim_time<=plot_end_date_obj)
sim_plot = sim_irf[sim_mask]
t_plot=[mpl.dates.date2num(d) for d in sim_time[sim_mask]] 

obs, obs_time = read_obs(obs_file)
obs_mask = (obs_time>=plot_start_date_obj) & (obs_time<=plot_end_date_obj)
obs_plot = obs[obs_mask]

title_str = '('+chr(ord(alpha)) +') No calibration\n(With summa defaults before sensitivity test)'
obs_fmt = 'k-'
sim_fmt = 'b--'

raw_plot = plot_hyd(t_plot, obs_plot, sim_plot, obs_fmt, sim_fmt, title_str, y_lim, ofile)
fig_file_summary.append(raw_plot)
del sim_irf, sim_time, t_plot

#====================================================================================#
# raw results (no calibration, after Andy's manual modification to summa)
raw_outputfile = 'raw.png'
fig_count = fig_count + 1
ofile = os.path.join(output_dir,raw_outputfile)
print('raw')

sim_irf, sim_time = read_sim(os.path.join(raw_dir, 'route/output/all.nc'),q_vname2,t_vname,time_format)
sim_mask = (sim_time>=plot_start_date_obj) & (sim_time<=plot_end_date_obj)
sim_plot = sim_irf[sim_mask]
t_plot=[mpl.dates.date2num(d) for d in sim_time[sim_mask]] 

obs, obs_time = read_obs(obs_file)
obs_mask = (obs_time>=plot_start_date_obj) & (obs_time<=plot_end_date_obj)
obs_plot = obs[obs_mask]

title_str = '('+chr(ord(alpha) + fig_count) +") No calibration\n(With updated summa defaults after sensitivity test)"
obs_fmt = 'k-'
sim_fmt = 'b--'

raw_plot = plot_hyd(t_plot, obs_plot, sim_plot, obs_fmt, sim_fmt, title_str, y_lim, ofile)
fig_file_summary.append(raw_plot)
del sim_irf, sim_time, t_plot

#====================================================================================#
# calibration results
for i in range(calib_num):
# for i in [0]:
    calib_outputfile = 'calib_trial'+str(i+1)+'.png'
    ofile = os.path.join(output_dir,calib_outputfile)
    calib_dir = os.path.join(root_dir,calib_dir_basename+str(i+1))
    print('calib trial '+str(i+1))
    
    sim_irf, sim_time = read_sim(os.path.join(calib_dir, sim_file),q_vname2,t_vname,time_format)
    sim_mask = (sim_time>=plot_start_date_obj) & (sim_time<=plot_end_date_obj)
    sim_plot = sim_irf[sim_mask]
    t_plot=[mpl.dates.date2num(d) for d in sim_time[sim_mask]] 

    fig_count = fig_count + 1
    title_str = '('+chr(ord(alpha) + fig_count) +') Calibration trial '+str(i+1)+' ('+objs[i]+') with route'
    obs_fmt = 'k-'
    sim_fmt = 'r--'

    calib_plot = plot_hyd(t_plot, obs_plot, sim_plot, obs_fmt, sim_fmt, title_str, y_lim, ofile)
    fig_file_summary.append(calib_plot)
    del sim_irf, sim_time, t_plot, calib_outputfile, ofile

#====================================================================================#
# save as one figure for all
print('save')
widths = []
heights = []
for fig_file in fig_file_summary:
    im = Image.open(os.path.join(output_dir, fig_file))
    widths.append(im.width)
    heights.append(im.height)

max_width = max(widths)
total_height = sum(heights)
new_im = Image.new('RGB', (max_width, total_height))

x_offset = 0
for fig_file in fig_file_summary:
    im = Image.open(os.path.join(output_dir, fig_file))    
    new_im.paste(im, (0,x_offset))
    x_offset += im.size[1]
new_im.save(os.path.join(output_dir, outputfile))

#====================================================================================#
# merge calib_var10_trial* with calib_route_trial* hydrographs
widths = []
heights = []
fig1=os.path.join(root_dir,'analysis/step1_plot_hydro/hydrgraph_calib_var10_w_orign.png')
fig2=os.path.join(root_dir,'analysis/step2_plot_route_hydro/hydrgraph_calib_route_w_orign.png')

for fig_file in [fig1,fig2]:
    im = Image.open(fig_file)
    widths.append(im.width)
    heights.append(im.height)

max_width = sum(widths)
total_height = max(heights)
new_im = Image.new('RGB', (max_width, total_height))

x_offset = 0
for fig_file in [fig1,fig2]:
    im = Image.open(fig_file)    
#     new_im.paste(im, (0,x_offset))
#     x_offset += im.size[1]
    new_im.paste(im, (x_offset,0))
    x_offset += im.size[0]
new_im.save(os.path.join(root_dir,'analysis/step2_plot_route_hydro/hydrograph_compare.png'))
print('Done')

raw_origin
raw
calib trial 1
calib trial 2
calib trial 3
calib trial 4
calib trial 5
calib trial 6
save
Done
